<a href="https://colab.research.google.com/github/CagataySencan/StressAnalysis/blob/main/Reddit_Stress_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Suppor Vector Machine


In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import  GaussianNB
from sklearn.metrics import  confusion_matrix
from sklearn.preprocessing import LabelEncoder
from gensim.parsing.preprocessing import remove_stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics
import seaborn as sb

In [2]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download ruchi798/stress-analysis-in-social-media

In [ ]:
! unzip stress-analysis-in-social-media.zip

In [8]:
trainData = pd.read_csv("dreaddit-train.csv")
testData = pd.read_csv(("dreaddit-test.csv"))

In [ ]:
# Eksik veri kontrolü
print("-Eğitim verisi-\n",trainData.isnull().sum(),"\n-------------\n","-Test Verisi-\n",testData.isnull().sum())

In [140]:
# Train ve test datalarımızı birleştirdik
X_Train = pd.DataFrame(trainData['text'], columns = ['text'])
X_Test = pd.DataFrame(testData['text'], columns = ['text'])
X = pd.concat((X_Train, X_Test), sort=False).reset_index(drop=True)

Y_Train =  pd.DataFrame(trainData['subreddit'], columns = ['subreddit'])
Y_Test =  pd.DataFrame(testData['subreddit'], columns = ['subreddit'])
Y =  pd.concat((Y_Train, Y_Test), sort=False).reset_index(drop=True)

In [ ]:
# Önişleme
nltk.download('wordnet')


In [ ]:
# Küçük harfe çevirme, stopwordslerden ve noktalama işaretlerinden temizleme
for i in range(X['text'].count()) : 
   
   text = X['text'][i]
   text = text.lower()
   text = remove_stopwords(text)
   text = text.translate(str.maketrans('','', punctuation))
   X['text'][i] = text

# Common wordlerden temizleme
cnt = Counter()
for text in X['text'].values : 
  for word in text.split() :
    cnt[word] += 1

cnt.most_common(10)




In [ ]:
freq = set([w for (w, wc) in cnt.most_common(10)])
def removeFreqwords(text):
    return " ".join([word for word in str(text).split() if word not in freq])

X['text'] = X['text'].apply(lambda text: removeFreqwords(text))

In [ ]:
# Lemmalama işlemi

lm = WordNetLemmatizer()

def lemmatizer(text):
    return " ".join([lm.lemmatize(word) for word in text.split()])

X['text'] = X['text'].apply(lambda text: lemmatizer(text))


In [ ]:
# Label Encoder ile subreddit label oluşturma
le = LabelEncoder()
subreddit_id = pd.DataFrame(le.fit_transform(Y), columns = ['subreddit_id'])
X['subreddit_id'] = subreddit_id


In [145]:
# Vektörleme işlemi
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english')

features = tfidf.fit_transform(X.text).toarray()

labels = X.subreddit_id

In [146]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25,random_state = 20)

In [167]:
model = LinearSVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Karmaşıklık matrisi :\n", confusion_matrix(y_test, y_pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Doğruluk skoru : \n", accuracy_score(y_test, y_pred))

# Recall
from sklearn.metrics import recall_score
print("Duyarlılık skoru : \n",recall_score(y_test, y_pred, average=None))

# Precision
from sklearn.metrics import precision_score
print("Kesinlik skoru : \n",precision_score(y_test, y_pred, average=None))

Karmaşıklık matrisi :
 [[  4   3   8   1   0   6   7   3   0   0]
 [  0  97   8   8   0   2  25  23   0   4]
 [  2   5  48   4   0   5   5  10   0   1]
 [  2   3   7  31   0   0  26  19   1   6]
 [  0   0   2   0   3   1   1   0   0   0]
 [  2   4   5   3   0  28   5   5   0   3]
 [  0  23   4   6   0   2 100  23   0   6]
 [  0  17   4  11   0   0  19 131   1   3]
 [  1   7   4   0   0   0   5   6   2   3]
 [  0   1   4   8   0   0  20  21   1  20]]
Doğruluk skoru : 
 0.5219347581552306
Duyarlılık skoru : 
 [0.125      0.58083832 0.6        0.32631579 0.42857143 0.50909091
 0.6097561  0.70430108 0.07142857 0.26666667]
Kesinlik skoru : 
 [0.36363636 0.60625    0.5106383  0.43055556 1.         0.63636364
 0.46948357 0.54356846 0.4        0.43478261]
